## Review top "non-bot" contributors in 2019

https://phabricator.wikimedia.org/T244925#5874567


From this analysis we want to get a sense for how prevalent unidentified bots are among our editors.

In [1]:
import requests
import pandas as pd
import json
import matplotlib.pyplot as plt
import gzip
from wmfdata import hive
import numpy as np

You can find the source for `wmfdata` at https://github.com/neilpquinn/wmfdata


### Total non-Bot Edits

In [5]:
total_nonbot_edits = hive.run("""
    SELECT
        wiki_db,
        COUNT(*) as edits    
    FROM wmf.mediawiki_history
    WHERE
        snapshot = "2020-01" AND
        event_timestamp between "2019-01-01" and "2020-01-01" AND
        event_entity = "revision" AND
        event_type = "create" AND
        size(event_user_is_bot_by) = 0 AND size(event_user_is_bot_by_historical) = 0
    GROUP BY wiki_db
""")

In [8]:
top_100_editors = hive.run("""
    SELECT 
        user_name,
        edits
    FROM
    (
        SELECT
            event_user_text as user_name,
            COUNT(*) as edits    
        FROM wmf.mediawiki_history
        WHERE
            snapshot = "2020-01" AND
            event_timestamp between "2019-01-01" and "2020-01-01" AND
            event_entity = "revision" AND
            event_type = "create" AND
            size(event_user_is_bot_by) = 0 AND size(event_user_is_bot_by_historical) = 0
        GROUP BY event_user_text
        HAVING edits >= 100000
    ) a
    ORDER BY edits DESC
    LIMIT 100
""")

In [27]:
top_100_editors.head(30)

,user_name,edits
0,Renamerr,19168338
1,Simon Villeneuve,10715795
2,Daniel Mietchen,9349361
3,Hogü-456,8475033
4,Ghuron,6311570
5,Florentyna,3991415
6,Sic19,3708443
7,&beer&love,3423374
8,Tagishsimon,2518952
9,Arpyia,2472691


In [18]:
print("Top 100 editors created " + str(top_100_editors.edits.sum()) +" edits in 2019. Which account for " + str(round(top_100_editors.edits.sum()/total_nonbot_edits.edits.sum()*100,2)) + " % of total non-bot edits.")

Top 100 editors created 125640683 edits in 2019. Which account for 42.79 % of total non-bot edits.


### Non-Bot Edits Excluding Wikidata

In [31]:
top_100_editors_wodata = hive.run("""
    SELECT 
        user_name,
        edits
    FROM
    (
        SELECT
            event_user_text as user_name,
            COUNT(*) as edits    
        FROM wmf.mediawiki_history
        WHERE
            snapshot = "2020-01" AND
            event_timestamp between "2019-01-01" and "2020-01-01" AND
            event_entity = "revision" AND
            event_type = "create" AND
            size(event_user_is_bot_by) = 0 AND size(event_user_is_bot_by_historical) = 0 AND
            wiki_db != "wikidatawiki"
        GROUP BY event_user_text
        HAVING edits >= 50000
    ) a
    ORDER BY edits DESC
    LIMIT 100
""")

In [33]:
top_100_editors_wodata.head(30)

,user_name,edits
0,Wikimedia Commons Welcome,1060215
1,Ser Amantio di Nicolao,651160
2,BrownHairedGirl,644542
3,Tm,586564
4,DerHexer,558264
5,Fæ,530356
6,Rudolphous,517828
7,Jc86035,486148
8,Alexis Jazz,482359
9,Judgefloro,406142


In [36]:
print("Top 100 editors created " + str(top_100_editors_wodata.edits.sum()) +" edits in 2019. Which account for " + str(round(top_100_editors_wodata.edits.sum()/total_nonbot_edits.loc[total_nonbot_edits['wiki_db'] != 'wikidatawiki', 'edits'].sum()*100,2)) + " % of total non-bot edits.")

Top 100 editors created 18905560 edits in 2019. Which account for 12.63 % of total non-bot edits.


In [45]:
len(set(top_100_editors['user_name']).intersection(set(top_100_editors_wodata['user_name'])))

30